In [214]:
!pip install pyspark
!pip install findspark  
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [215]:
from pyspark.sql import SparkSession
import findspark
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import * 
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType  
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator



findspark.init()

spark = SparkSession.builder.appName("Pyspark Tutorial")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","10g")\
                    .getOrCreate()


# check spark configuration

spark

In [122]:
!curl https://raw.githubusercontent.com/Sepay/Programming/main/obesity.csv >> obesity.csv
    
df = spark.read.csv('obesity.csv',header=True,sep=",")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  257k  100  257k    0     0   463k      0 --:--:-- --:--:-- --:--:--  464k


In [130]:
df = df.withColumnRenamed('family_history_with_overweight', 'FH')

In [131]:
df.show(5,0)

+------+---+------+------+---+----+----+---+---------+-----+----+---+---+---+----------+---------------------+-------------------+
|Gender|Age|Height|Weight|FH |FAVC|FCVC|NCP|CAEC     |SMOKE|CH2O|SCC|FAF|TUE|CALC      |MTRANS               |NObeyesdad         |
+------+---+------+------+---+----+----+---+---------+-----+----+---+---+---+----------+---------------------+-------------------+
|Female|21 |1.62  |64    |yes|no  |2   |3  |Sometimes|no   |2   |no |0  |1  |no        |Public_Transportation|Normal_Weight      |
|Female|21 |1.52  |56    |yes|no  |3   |3  |Sometimes|yes  |3   |yes|3  |0  |Sometimes |Public_Transportation|Normal_Weight      |
|Male  |23 |1.8   |77    |yes|no  |2   |3  |Sometimes|no   |2   |no |2  |1  |Frequently|Public_Transportation|Normal_Weight      |
|Male  |27 |1.8   |87    |no |no  |3   |3  |Sometimes|no   |2   |no |2  |0  |Frequently|Walking              |Overweight_Level_I |
|Male  |22 |1.78  |89.8  |no |no  |2   |1  |Sometimes|no   |2   |no |0  |0  |Someti

In [132]:
df.count()

40127

In [133]:
df= df.drop('TUE')

In [134]:
df.show()

+------+---+------+------+---+----+----+---+----------+-----+----+---+---+----------+--------------------+-------------------+
|Gender|Age|Height|Weight| FH|FAVC|FCVC|NCP|      CAEC|SMOKE|CH2O|SCC|FAF|      CALC|              MTRANS|         NObeyesdad|
+------+---+------+------+---+----+----+---+----------+-----+----+---+---+----------+--------------------+-------------------+
|Female| 21|  1.62|    64|yes|  no|   2|  3| Sometimes|   no|   2| no|  0|        no|Public_Transporta...|      Normal_Weight|
|Female| 21|  1.52|    56|yes|  no|   3|  3| Sometimes|  yes|   3|yes|  3| Sometimes|Public_Transporta...|      Normal_Weight|
|  Male| 23|   1.8|    77|yes|  no|   2|  3| Sometimes|   no|   2| no|  2|Frequently|Public_Transporta...|      Normal_Weight|
|  Male| 27|   1.8|    87| no|  no|   3|  3| Sometimes|   no|   2| no|  2|Frequently|             Walking| Overweight_Level_I|
|  Male| 22|  1.78|  89.8| no|  no|   2|  1| Sometimes|   no|   2| no|  0| Sometimes|Public_Transporta...|Overw

# Variáveis

Retirou-se a coluna dos telemoveis 

# TRATAMENTO DE DADOS

Alterou-se as colunas Gender, FH, CAEC, SMOLE, SCC, CALC, MTRANS, OBESITY para inteiros para ser mais facil executar os algoritmos


In [135]:
df.groupBy('NObeyesdad').count().show()

+-------------------+-----+
|         NObeyesdad|count|
+-------------------+-----+
|   Obesity_Type_III| 6156|
| Overweight_Level_I| 5510|
|    Obesity_Type_II| 5643|
|Insufficient_Weight| 5168|
|Overweight_Level_II| 5510|
|      Normal_Weight| 5453|
|         NObeyesdad|   18|
|     Obesity_Type_I| 6669|
+-------------------+-----+



In [136]:


df = df.withColumn("CAEC", when(col("CAEC")== "Sometimes", 0).when(col("CAEC") == "Frequently", 1).otherwise(2))
df = df.withColumn("Gender", when(col("Gender")== "Female", 0).otherwise(1))
df = df.withColumn("FH", when(col("FH")== "yes", 0).otherwise(1))
df = df.withColumn("FAVC", when(col("FAVC")== "yes", 0).otherwise(1))
df = df.withColumn("SMOKE", when(col("SMOKE")== "yes", 0).otherwise(1))
df = df.withColumn("SCC", when(col("SCC")== "yes", 0).otherwise(1))
df = df.withColumn("CALC", when(col("CALC")== "no", 0).when(col("CALC") == "Sometimes", 1).when(col("CALC")=="Frequently", 2).otherwise(3))
df = df.withColumn("MTRANS", when(col("MTRANS")== "Public_Transportation", 0).when(col("MTRANS") == "Automobile", 0).when(col("MTRANS")=="Motorbike", 0).otherwise(1))
df = df.withColumn("NObeyesdad", when(col("NObeyesdad")== "Insufficient_Weight", 0).when(col("NObeyesdad") == "Normal_Weight", 1).when(col("NObeyesdad")=="Overweight_Level_I", 2).when(col("NObeyesdad")== "Overweight_Level_II",3).otherwise(4))

In [137]:
df = df.withColumnRenamed("family_history_with_overweight","Family_HIST")
df = df.withColumnRenamed('FAVC','CALORICFOOD')
df=df.withColumnRenamed('Gender','GENDER')
df=df.withColumnRenamed('Age','AGE')
df=df.withColumnRenamed('Weight','WEIGHT')
df=df.withColumnRenamed('Height','HEIGHT')
df=df.withColumnRenamed('FCVC','VEGET')
df=df.withColumnRenamed('NCP','MEALS')
df=df.withColumnRenamed('CAEC','BETWEENMEALS')
df=df.withColumnRenamed('CH2O','H20')
df=df.withColumnRenamed('SCC','CALORIES')
df=df.withColumnRenamed('FAF','PHYSY')
df=df.withColumnRenamed('CALC','ALCH')
df=df.withColumnRenamed('MTRANS','TRANSPORT')
df=df.withColumnRenamed('NObeyesdad','OBESITY')

In [138]:
df.show()

+------+---+------+------+---+-----------+-----+-----+------------+-----+---+--------+-----+----+---------+-------+
|GENDER|AGE|HEIGHT|WEIGHT| FH|CALORICFOOD|VEGET|MEALS|BETWEENMEALS|SMOKE|H20|CALORIES|PHYSY|ALCH|TRANSPORT|OBESITY|
+------+---+------+------+---+-----------+-----+-----+------------+-----+---+--------+-----+----+---------+-------+
|     0| 21|  1.62|    64|  0|          1|    2|    3|           0|    1|  2|       1|    0|   0|        0|      1|
|     0| 21|  1.52|    56|  0|          1|    3|    3|           0|    0|  3|       0|    3|   1|        0|      1|
|     1| 23|   1.8|    77|  0|          1|    2|    3|           0|    1|  2|       1|    2|   2|        0|      1|
|     1| 27|   1.8|    87|  1|          1|    3|    3|           0|    1|  2|       1|    2|   2|        1|      2|
|     1| 22|  1.78|  89.8|  1|          1|    2|    1|           0|    1|  2|       1|    0|   1|        0|      3|
|     1| 29|  1.62|    53|  1|          0|    2|    3|           0|    1

Converter tudo para int

In [139]:

df = df.withColumn("VEGET",col("VEGET").cast(IntegerType())) \
    .withColumn("MEALS",col("MEALS").cast(IntegerType())) \
    .withColumn("H20",col("H20").cast(IntegerType())) \
    .withColumn("PHYSY",col("PHYSY").cast(IntegerType()))
df = df.fillna(0)
df.printSchema()

root
 |-- GENDER: integer (nullable = false)
 |-- AGE: string (nullable = true)
 |-- HEIGHT: string (nullable = true)
 |-- WEIGHT: string (nullable = true)
 |-- FH: integer (nullable = false)
 |-- CALORICFOOD: integer (nullable = false)
 |-- VEGET: integer (nullable = true)
 |-- MEALS: integer (nullable = true)
 |-- BETWEENMEALS: integer (nullable = false)
 |-- SMOKE: integer (nullable = false)
 |-- H20: integer (nullable = true)
 |-- CALORIES: integer (nullable = false)
 |-- PHYSY: integer (nullable = true)
 |-- ALCH: integer (nullable = false)
 |-- TRANSPORT: integer (nullable = false)
 |-- OBESITY: integer (nullable = false)



In [140]:

assemble=VectorAssembler(inputCols=[
    'FH','CALORICFOOD', "SMOKE", "VEGET", "MEALS", 
    "BETWEENMEALS", "H20", "CALORIES", "PHYSY", "ALCH"
], outputCol='features')

In [141]:
assembled_data=assemble.transform(df)

In [142]:
scale=StandardScaler(inputCol='features',outputCol='standardized')
scale=scale.fit(assembled_data)
scale_output= scale.transform(assembled_data)

scale_output.select('standardized').show(2,truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------+
|standardized                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------+
|[0.0,3.117459138024327,7.0013469662372225,3.3280190253331385,3.607354980448749,0.0,3.3356217688990903,4.800676795202441,0.0,0.0]|
|[0.0,3.117459138024327,0.0,4.992028537999708,3.607354980448749,0.0,5.003432653348636,0.0,3.603241561264033,1.9323627596117945]  |
+--------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [158]:
finalData = scale_output.select("features", "OBESITY")
train, test =finalData.randomSplit([0.8,0.2])

In [185]:

lr = LogisticRegression(featuresCol="features" ,labelCol="OBESITY", maxIter=5)
lrModel = lr.fit(train)

In [193]:
predictions = lrModel.transform(test)
predictions.select("OBESITY", "features", "rawPrediction", "prediction", "probability").toPandas().head(5)

,OBESITY,features,rawPrediction,prediction,probability
0,0,"(1.0, 0.0, 1.0, 3.0, 2.0, 0.0, 2.0, 0.0, 0.0, ...","[1.3748497045543018, 0.7366174900550444, 1.238...",0.0,"[0.4007803684874383, 0.2117023657842629, 0.349..."
1,0,"(1.0, 0.0, 1.0, 3.0, 2.0, 0.0, 2.0, 0.0, 0.0, ...","[1.3748497045543018, 0.7366174900550444, 1.238...",0.0,"[0.4007803684874383, 0.2117023657842629, 0.349..."
2,0,"(1.0, 0.0, 1.0, 3.0, 2.0, 0.0, 2.0, 0.0, 0.0, ...","[1.3748497045543018, 0.7366174900550444, 1.238...",0.0,"[0.4007803684874383, 0.2117023657842629, 0.349..."
3,0,"(1.0, 0.0, 1.0, 3.0, 2.0, 0.0, 2.0, 0.0, 0.0, ...","[1.3748497045543018, 0.7366174900550444, 1.238...",0.0,"[0.4007803684874383, 0.2117023657842629, 0.349..."
4,4,"(0.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 1.0, 0.0, ...","[-1.9390108450789176, 0.3997760068412246, -1.8...",3.0,"[0.010861034195427216, 0.11261426461744567, 0...."


In [190]:


eval = BinaryClassificationEvaluator(rawPredictionCol = "prediction", labelCol = "OBESITY")
auc = eval.evaluate(predictions)
print(auc)

0.7680613360789001


In [204]:
FP = predictions.where(predictions["OBESITY"] == predictions["prediction"]).count()
accuracy = FP /float(predictions.count())
accuracy

0.6066028225806451

In [206]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: OBESITY)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. T

In [213]:

paramGrid = (ParamGridBuilder()
            .addGrid(lr.regParam[0.01,0.5,2.0])
            .addGrid(lr.elasticNetParam[0.0,0.5,1.0])
            .addGrid(lr.maxIter[1,5,10])
            .build())

TypeError: 'Param' object is not subscriptable